In [1]:
import cython

In [2]:
%load_ext cython

In [5]:
%%cython -a -f
# distutils: language = c
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False


import numpy as np

from cython.cimports.cpython.mem import PyMem_Malloc, PyMem_Realloc, PyMem_Free
from CyRK.utils cimport allocate_mem

cdef struct sub_array:
    Py_ssize_t start_index
    Py_ssize_t size
    double* array_ptr
    sub_array* head
    sub_array* next_array


cdef llarray_expand(sub_array* last_array, Py_ssize_t expansion_size):
    
    if sub_array.next_array != NULL:
        raise AttributeError("Can only expand the last sub array of a linked list.")
    
    # Create a new sub-array on the stack
    cdef sub_array new_sub_array
    
    # Allocate its actual array data on the heap
    new_sub_array.array_ptr = <double *> allocate_mem(expansion_size * sizeof(double), "Linked-List-Array:: expansion (llarray_expand).")
    
    # Record linked list information.
    new_sub_array.size = expansion_size
    new_sub_array.start_index = last_array.start_index + last_array.size
    # Head should always point to the first array.
    new_sub_array.head = last_array.head
    # There is no next array yet. Set to Null
    new_sub_array.next_array = NULL
    
    # Set old array pointer to this new sub array
    last_array[0] = &new_sub_array

    
cdef double* llarray_collapse(sub_array* last_array):
    
    cdef Py_ssize_t i, j, start_index
    
    cdef ssize_t num_arrays = 0
    cdef Py_ssize_t total_size = 0
    cdef Py_ssize_t subarray_size = 0
    cdef sub_array* head_subarray_ptr = sub_array.head
    cdef sub_array* temp_subarray_ptr = head_subarray_ptr
    cdef sub_array* next_subarray_ptr = head_subarray_ptr
    
    cdef double* output_array_ptr
    
    if sub_array.head == NULL:
        raise AttributeError("LLArray has no head! Can not collapse.")
    
    # Step through linked list of arrays and count how many arrays are there and the total size.
    while True:
        
        if next_subarray_ptr == NULL:
            # At the end of list
            break
        
        # Get size information
        total_size += next_subarray_ptr.size
        
        # Setup for next subarray
        next_subarray_ptr = next_subarray_ptr.next_array
        num_arrays += 1
    
    # Now that we have the required data we can perform the collapse
    # First, the special case that there is only one array
    if num_arrays == 1:
        # Only head subarray
        output_array_ptr = head_subarray_ptr.array_ptr
    else:
        # Need to allocate new memory on the heap
        output_array_ptr = <double *> allocate_mem(total_size * sizeof(double), "Linked-List-Array:: Output Array (llarray_collapse).")
        
        # Step through each subarray and pull out the data
        next_subarray_ptr = head_subarray_ptr
        for i in range(num_arrays):
            subarray_size = next_subarray_ptr.size
            start_index = next_subarray_ptr.start_index
            
            for j in range(subarray_size): ## OPT: prange
                output_array_ptr[start_index + j] = next_subarray_ptr.array_ptr[j]
            
            temp_subarray_ptr = next_subarray_ptr.next_array
            
            # We need to free the sub arrays
            next_subarray_ptr.next_array = NULL
            next_subarray_ptr.head = NULL
            if next_subarray_ptr.array_ptr != NULL:
                PyMem_Free(next_subarray_ptr.array_ptr)
            
    
    return output_array_ptr

    

x = np.linspace(0., 10., 10, dtype=np.float64)
cdef double[:] x_view = x

y = np.linspace(10., 20., 5, dtype=np.float64)
cdef double[:] y_view = y

cdef sub_array a
a.size = 10
a.array_ptr = &x_view[0]
a.next_array = NULL

cdef sub_array b
b.size = 5
b.array_ptr = &y_view[0]
b.next_array = NULL

a.next_array = &b


for i in range(15):
    if i < 10:
        print(a.array_ptr[i])
    else:
        i2 = i-10
        print(a.next_array.array_ptr[i2])
    


Content of stdout:
_cython_magic_e994992728716088c1940f730d0d236ec42c399f.c
C:\Users\jrenaud\.ipython\cython\_cython_magic_e994992728716088c1940f730d0d236ec42c399f.c(1468): warning C4005: '__pyx_nonatomic_int_type': macro redefinition
C:\Users\jrenaud\.ipython\cython\_cython_magic_e994992728716088c1940f730d0d236ec42c399f.c(1419): note: see previous definition of '__pyx_nonatomic_int_type'
   Creating library C:\Users\jrenaud\.ipython\cython\Users\jrenaud\.ipython\cython\_cython_magic_e994992728716088c1940f730d0d236ec42c399f.cp311-win_amd64.lib and object C:\Users\jrenaud\.ipython\cython\Users\jrenaud\.ipython\cython\_cython_magic_e994992728716088c1940f730d0d236ec42c399f.cp311-win_amd64.exp
Generating code
Finished generating code0.0
1.1111111111111112
2.2222222222222223
3.3333333333333335
4.444444444444445
5.555555555555555
6.666666666666667
7.777777777777779
8.88888888888889
10.0
10.0
12.5
15.0
17.5
20.0


In [8]:
%%cython -a -f
# distutils: language = c
# cython: boundscheck=False, wraparound=False, nonecheck=False, cdivision=True, initializedcheck=False

cdef struct SubArray:
    Py_ssize_t start_index
    Py_ssize_t size
    double* array_ptr

cdef SubArray x
print()

print(sizeof(SubArray))

print(sizeof(x))

print()

cdef SubArray xL[1000]

print(sizeof(xL))

Content of stdout:
_cython_magic_020a7c2d769d46035ae96294d669fb57aa824f1f.c
   Creating library C:\Users\jrenaud\.ipython\cython\Users\jrenaud\.ipython\cython\_cython_magic_020a7c2d769d46035ae96294d669fb57aa824f1f.cp311-win_amd64.lib and object C:\Users\jrenaud\.ipython\cython\Users\jrenaud\.ipython\cython\_cython_magic_020a7c2d769d46035ae96294d669fb57aa824f1f.cp311-win_amd64.exp
Generating code
Finished generating code
24
24

24000


In [4]:
a

NameError: name 'a' is not defined